In [1]:
import numpy as np
import random
from collections import defaultdict
import pickle  # For saving the model

In [2]:
# b) Defining the Tic Tac Toe game
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 0: empty, 1: X, -1: O
        self.current_player = 1  # Start with X (1)

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        return self.get_state()

    def get_state(self):
        # Flatten the board to a tuple for hashing
        return tuple(self.board.flatten())

    def available_actions(self):
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def make_move(self, action):
        i, j = action
        if self.board[i, j] == 0:
            self.board[i, j] = self.current_player
            self.current_player = -self.current_player  # Switch player
            return True
        return False

    def check_winner(self):
        # Check rows, columns, diagonals
        for i in range(3):
            if abs(np.sum(self.board[i, :])) == 3:
                return self.board[i, 0]
            if abs(np.sum(self.board[:, i])) == 3:
                return self.board[0, i]
        if abs(np.trace(self.board)) == 3 or abs(self.board[::-1].trace()) == 3:
            return self.board[1, 1] if self.board[1, 1] != 0 else 0
        return 0

    def is_done(self):
        winner = self.check_winner()
        if winner != 0:
            return True, winner
        if len(self.available_actions()) == 0:
            return True, 0  # Draw
        return False, 0

    def print_board(self):
        symbols = {1: 'X', -1: 'O', 0: ' '}
        for row in self.board:
            print(' | '.join(symbols[cell] for cell in row))
            print('-' * 5)

In [3]:
# c) Building the reinforcement learning model
# Using Q-learning with a tabular approach. Q-table indexed by state-action pairs.

class QLearningAgent:
    def __init__(self, player=1, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.player = player
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate

    def get_action(self, state, available_actions):
        state_tuple = tuple(state.flatten())
        if random.random() < self.epsilon:
            return random.choice(available_actions)
        q_values = self.q_table[state_tuple]
        best_action = max(available_actions, key=lambda a: q_values[(a[0], a[1])])
        return best_action

    def update(self, state, action, reward, next_state, done):
        state_tuple = tuple(state.flatten())
        next_state_tuple = tuple(next_state.flatten())
        old_value = self.q_table[state_tuple][action]
        if done:
            target = reward
        else:
            available_next = TicTacToe().available_actions()  # Approximate, but in practice, pass it
            # For simplicity, we assume opponent plays randomly or fixed, but here we max over possible
            next_q = max(self.q_table[next_state_tuple].values()) if self.q_table[next_state_tuple] else 0
            target = reward + self.gamma * next_q
        new_value = old_value + self.alpha * (target - old_value)
        self.q_table[state_tuple][action] = new_value

    def save_model(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(dict(self.q_table), f)

    def load_model(self, filename):
        try:
            with open(filename, 'rb') as f:
                self.q_table = defaultdict(lambda: defaultdict(float), pickle.load(f))
        except FileNotFoundError:
            pass

In [4]:
# d) Training the model
def train_model(episodes=10000):
    env = TicTacToe()
    agent_x = QLearningAgent(player=1)
    agent_o = QLearningAgent(player=-1)  # Opponent also learns, self-play

    for episode in range(episodes):
        state = env.reset()
        done = False
        while not done:
            if env.current_player == 1:
                agent = agent_x
                opp_agent = agent_o
            else:
                agent = agent_o
                opp_agent = agent_x

            available = env.available_actions()
            action = agent.get_action(env.board, available)
            old_state = env.board.copy()
            env.make_move(action)
            next_state = env.board.copy()

            done, reward = env.is_done()
            if done:
                if reward == agent.player:
                    reward = 1
                elif reward == -agent.player:
                    reward = -1
                else:
                    reward = 0
            else:
                # Small negative for continuing
                reward = -0.01

            agent.update(old_state, action, reward, next_state, done)

            # Opponent's turn is handled in the loop

        if episode % 1000 == 0:
            print(f"Episode {episode} completed.")

    agent_x.save_model('x_agent.pkl')
    agent_o.save_model('o_agent.pkl')
    return agent_x, agent_o

In [5]:
# e) Testing the model
def test_model(agent_x, agent_o, num_games=10):
    env = TicTacToe()
    wins_x, wins_o, draws = 0, 0, 0

    for game in range(num_games):
        state = env.reset()
        done = False
        print(f"\nGame {game + 1}:")
        while not done:
            if env.current_player == 1:
                agent = agent_x
            else:
                agent = agent_o

            available = env.available_actions()
            # Use greedy policy for testing (epsilon=0)
            state_tuple = tuple(env.board.flatten())
            if agent.q_table[state_tuple]:
                action = max(available, key=lambda a: agent.q_table[state_tuple][a])
            else:
                action = random.choice(available)

            env.make_move(action)
            env.print_board()

            done, winner = env.is_done()
            if done:
                if winner == 1:
                    wins_x += 1
                    print("X wins!")
                elif winner == -1:
                    wins_o += 1
                    print("O wins!")
                else:
                    draws += 1
                    print("Draw!")

    print(f"\nResults after {num_games} games:")
    print(f"X wins: {wins_x}, O wins: {wins_o}, Draws: {draws}")


In [6]:
# Run training and testing
if __name__ == "__main__":
    print("Training the model...")
    agent_x, agent_o = train_model(5000)  # Reduced episodes for demo

    print("\nLoading models for testing...")
    # In practice, load if saved
    # agent_x.load_model('x_agent.pkl')
    # agent_o.load_model('o_agent.pkl')

    print("Testing the model...")
    test_model(agent_x, agent_o, 5)

Training the model...
Episode 0 completed.
Episode 1000 completed.
Episode 2000 completed.
Episode 3000 completed.
Episode 4000 completed.

Loading models for testing...
Testing the model...

Game 1:
X |   |  
-----
  |   |  
-----
  |   |  
-----
X |   |  
-----
  |   |  
-----
  |   | O
-----
X |   |  
-----
  |   |  
-----
  | X | O
-----
X |   |  
-----
  | O |  
-----
  | X | O
-----
X |   |  
-----
  | O |  
-----
X | X | O
-----
X |   |  
-----
  | O | O
-----
X | X | O
-----
X |   | X
-----
  | O | O
-----
X | X | O
-----
X |   | X
-----
O | O | O
-----
X | X | O
-----
O wins!

Game 2:
X |   |  
-----
  |   |  
-----
  |   |  
-----
X |   |  
-----
  |   |  
-----
  |   | O
-----
X |   |  
-----
  |   |  
-----
  | X | O
-----
X |   |  
-----
  | O |  
-----
  | X | O
-----
X |   |  
-----
  | O |  
-----
X | X | O
-----
X |   |  
-----
  | O | O
-----
X | X | O
-----
X |   |  
-----
X | O | O
-----
X | X | O
-----
X wins!

Game 3:
X |   |  
-----
  |   |  
-----
  |   |  
----

In [7]:
import numpy as np
import random
from collections import defaultdict
import pickle  # For saving the model
from IPython.display import Markdown, display  # For better formatting in Jupyter, but for console, we'll use print

# a) Setting up the environment
# We use numpy for array operations, random for exploration, and defaultdict for Q-table.
# No external libraries beyond basics are needed.

# b) Defining the Tic Tac Toe game
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 0: empty, 1: X, -1: O
        self.current_player = 1  # Start with X (1)

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        return self.get_state()

    def get_state(self):
        # Flatten the board to a tuple for hashing
        return tuple(self.board.flatten())

    def available_actions(self):
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def make_move(self, action):
        i, j = action
        if self.board[i, j] == 0:
            self.board[i, j] = self.current_player
            self.current_player = -self.current_player  # Switch player
            return True
        return False

    def check_winner(self):
        # Check rows, columns, diagonals
        for i in range(3):
            if abs(np.sum(self.board[i, :])) == 3:
                return self.board[i, 0]
            if abs(np.sum(self.board[:, i])) == 3:
                return self.board[0, i]
        if abs(np.trace(self.board)) == 3 or abs(self.board[::-1].trace()) == 3:
            return self.board[1, 1] if self.board[1, 1] != 0 else 0
        return 0

    def is_done(self):
        winner = self.check_winner()
        if winner != 0:
            return True, winner
        if len(self.available_actions()) == 0:
            return True, 0  # Draw
        return False, 0

    def print_board_markdown(self):
        symbols = {1: 'X', -1: 'O', 0: ' '}
        board_str = "|   |   |   |\n"
        board_str += "|---|---|---|\n"
        for i in range(3):
            row = "| " + " | ".join(symbols[self.board[i, j]] for j in range(3)) + " |\n"
            board_str += row
            if i < 2:
                board_str += "|---|---|---|\n"
        print(board_str)

# c) Building the reinforcement learning model
# Using Q-learning with a tabular approach. Q-table indexed by state-action pairs.

class QLearningAgent:
    def __init__(self, player=1, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.player = player
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate

    def get_action(self, state, available_actions):
        state_tuple = tuple(state.flatten())
        if random.random() < self.epsilon:
            return random.choice(available_actions)
        q_values = self.q_table[state_tuple]
        best_action = max(available_actions, key=lambda a: q_values[(a[0], a[1])])
        return best_action

    def update(self, state, action, reward, next_state, done):
        state_tuple = tuple(state.flatten())
        next_state_tuple = tuple(next_state.flatten())
        old_value = self.q_table[state_tuple][action]
        if done:
            target = reward
        else:
            next_q = max(self.q_table[next_state_tuple].values()) if self.q_table[next_state_tuple] else 0
            target = reward + self.gamma * next_q
        new_value = old_value + self.alpha * (target - old_value)
        self.q_table[state_tuple][action] = new_value

    def save_model(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(dict(self.q_table), f)

    def load_model(self, filename):
        try:
            with open(filename, 'rb') as f:
                self.q_table = defaultdict(lambda: defaultdict(float), pickle.load(f))
        except FileNotFoundError:
            pass

# d) Training the model
def train_model(episodes=10000):
    env = TicTacToe()
    agent_x = QLearningAgent(player=1)
    agent_o = QLearningAgent(player=-1)  # Opponent also learns, self-play

    for episode in range(episodes):
        state = env.reset()
        done = False
        while not done:
            if env.current_player == 1:
                agent = agent_x
                opp_agent = agent_o
            else:
                agent = agent_o
                opp_agent = agent_x

            available = env.available_actions()
            action = agent.get_action(env.board, available)
            old_state = env.board.copy()
            env.make_move(action)
            next_state = env.board.copy()

            done, reward = env.is_done()
            if done:
                if reward == agent.player:
                    reward = 1
                elif reward == -agent.player:
                    reward = -1
                else:
                    reward = 0
            else:
                # Small negative for continuing
                reward = -0.01

            agent.update(old_state, action, reward, next_state, done)

        if episode % 1000 == 0:
            print(f"Episode {episode} completed.")

    agent_x.save_model('x_agent.pkl')
    agent_o.save_model('o_agent.pkl')
    return agent_x, agent_o

# e) Testing the model
def test_model(agent_x, agent_o, num_games=5):
    wins_x, wins_o, draws = 0, 0, 0

    for game in range(num_games):
        env = TicTacToe()
        done = False
        print(f"\n### Game {game + 1}:")
        move_count = 0
        while not done:
            if env.current_player == 1:
                agent = agent_x
            else:
                agent = agent_o

            available = env.available_actions()
            # Use greedy policy for testing (epsilon=0)
            state_tuple = tuple(env.board.flatten())
            if agent.q_table[state_tuple]:
                action = max(available, key=lambda a: agent.q_table[state_tuple][a])
            else:
                action = random.choice(available)

            env.make_move(action)
            move_count += 1
            print(f"\n**Move {move_count}:**")
            env.print_board_markdown()

            done, winner = env.is_done()
            if done:
                if winner == 1:
                    wins_x += 1
                    print("**X wins!**")
                elif winner == -1:
                    wins_o += 1
                    print("**O wins!**")
                else:
                    draws += 1
                    print("**Draw!**")

    print(f"\n### Final Results after {num_games} games:")
    print(f"**X wins: {wins_x}**, **O wins: {wins_o}**, **Draws: {draws}**")

# Run training and testing
if __name__ == "__main__":
    print("Training the model...")
    agent_x, agent_o = train_model(5000)  # Reduced episodes for demo

    print("\nTesting the model...")
    test_model(agent_x, agent_o, 5)

Training the model...
Episode 0 completed.
Episode 1000 completed.
Episode 2000 completed.
Episode 3000 completed.
Episode 4000 completed.

Testing the model...

### Game 1:

**Move 1:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   |   |   |
|---|---|---|
|   |   |   |


**Move 2:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   |   |   |
|---|---|---|
|   | O |   |


**Move 3:**
|   |   |   |
|---|---|---|
| X |   | X |
|---|---|---|
|   |   |   |
|---|---|---|
|   | O |   |


**Move 4:**
|   |   |   |
|---|---|---|
| X |   | X |
|---|---|---|
|   |   |   |
|---|---|---|
| O | O |   |


**Move 5:**
|   |   |   |
|---|---|---|
| X | X | X |
|---|---|---|
|   |   |   |
|---|---|---|
| O | O |   |

**X wins!**

### Game 2:

**Move 1:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   |   |   |
|---|---|---|
|   |   |   |


**Move 2:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   |   |   |
|---|---|---|
|   | O |   |


**Move 3:**
|   

In [13]:
import numpy as np
import random
from collections import defaultdict
import pickle  # For saving the model

# a) Setting up the environment
# We use numpy for array operations, random for exploration, and defaultdict for Q-table.
# No external libraries beyond basics are needed.

# b) Defining the Tic Tac Toe game
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 0: empty, 1: X, -1: O
        self.current_player = 1  # Start with X (1)

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        return self.get_state()

    def get_state(self):
        # Flatten the board to a tuple for hashing
        return tuple(self.board.flatten())

    def available_actions(self):
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def make_move(self, action):
        i, j = action
        if self.board[i, j] == 0:
            self.board[i, j] = self.current_player
            self.current_player = -self.current_player  # Switch player
            return True
        return False

    def check_winner(self):
        # Check rows, columns, diagonals
        for i in range(3):
            if abs(np.sum(self.board[i, :])) == 3:
                return self.board[i, 0]
            if abs(np.sum(self.board[:, i])) == 3:
                return self.board[0, i]
        if abs(np.trace(self.board)) == 3 or abs(self.board[::-1].trace()) == 3:
            return self.board[1, 1] if self.board[1, 1] != 0 else 0
        return 0

    def is_done(self):
        winner = self.check_winner()
        if winner != 0:
            return True, winner
        if len(self.available_actions()) == 0:
            return True, 0  # Draw
        return False, 0

    def print_board_markdown(self):
        symbols = {1: 'X', -1: 'O', 0: ' '}
        board_str = "|   |   |   |\n"
        board_str += "|---|---|---|\n"
        for i in range(3):
            row = "| " + " | ".join(symbols[self.board[i, j]] for j in range(3)) + " |\n"
            board_str += row
            if i < 2:
                board_str += "|---|---|---|\n"
        print(board_str)

# Random Agent for testing against random opponent
class RandomAgent:
    def get_action(self, state, available_actions):
        return random.choice(available_actions)

# c) Building the reinforcement learning model
# Using Q-learning with epsilon decay for better convergence and accuracy
class QLearningAgent:
    def __init__(self, player=1, alpha=0.1, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.player = player
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

    def get_action(self, state, available_actions):
        state_tuple = tuple(state.flatten())
        if random.random() < self.epsilon:
            return random.choice(available_actions)
        q_values = self.q_table[state_tuple]
        best_action = max(available_actions, key=lambda a: q_values[(a[0], a[1])])
        return best_action

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def update(self, state, action, reward, next_state, done):
        state_tuple = tuple(state.flatten())
        next_state_tuple = tuple(next_state.flatten())
        old_value = self.q_table[state_tuple][action]
        if done:
            target = reward
        else:
            next_q = max(self.q_table[next_state_tuple].values()) if self.q_table[next_state_tuple] else 0
            target = reward + self.gamma * next_q
        new_value = old_value + self.alpha * (target - old_value)
        self.q_table[state_tuple][action] = new_value

    def save_model(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(dict(self.q_table), f)

    def load_model(self, filename):
        try:
            with open(filename, 'rb') as f:
                self.q_table = defaultdict(lambda: defaultdict(float), pickle.load(f))
        except FileNotFoundError:
            pass

    def set_greedy(self):
        self.epsilon = 0  # For testing, no exploration

# d) Training the model with self-play and epsilon decay
def train_model(episodes=20000):
    env = TicTacToe()
    agent_x = QLearningAgent(player=1, epsilon=1.0)
    agent_o = QLearningAgent(player=-1, epsilon=1.0)  # Self-play for learning

    for episode in range(episodes):
        env.reset()
        done = False
        while not done:
            if env.current_player == 1:
                agent = agent_x
            else:
                agent = agent_o

            available = env.available_actions()
            if not available:
                break
            action = agent.get_action(env.board, available)
            old_state = env.board.copy()
            env.make_move(action)
            next_state = env.board.copy()

            done, reward = env.is_done()
            if done:
                if reward == agent.player:
                    reward = 1
                elif reward == -agent.player:
                    reward = -1
                else:
                    reward = 0
            else:
                reward = -0.01  # Small penalty for longer games

            agent.update(old_state, action, reward, next_state, done)

        agent_x.decay_epsilon()
        agent_o.decay_epsilon()

        if episode % 5000 == 0:
            print(f"Episode {episode} completed. Epsilon X: {agent_x.epsilon:.3f}, O: {agent_o.epsilon:.3f}")

    agent_x.save_model('x_agent.pkl')
    agent_o.save_model('o_agent.pkl')
    agent_x.set_greedy()
    agent_o.set_greedy()
    return agent_x

# e) Testing the model against a random opponent to evaluate accuracy
def test_model_vs_random(agent_x, num_games=10):
    wins_x, wins_o, draws = 0, 0, 0
    random_agent = RandomAgent()

    for game in range(num_games):
        env = TicTacToe()
        random.seed(game)  # Seed for reproducibility and variety
        np.random.seed(game)
        done = False
        print(f"\n### Game {game + 1} vs Random O:")
        move_count = 0
        while not done:
            if env.current_player == 1:
                agent = agent_x
            else:
                agent = random_agent

            available = env.available_actions()
            if not available:
                break
            if isinstance(agent, QLearningAgent):
                state_tuple = tuple(env.board.flatten())
                if agent.q_table[state_tuple]:
                    action = max(available, key=lambda a: agent.q_table[state_tuple][a])
                else:
                    action = random.choice(available)
            else:
                action = agent.get_action(env.board, available)

            env.make_move(action)
            move_count += 1
            # Print board after odd moves (X's turns) or at end
            if move_count % 2 == 1 or move_count >= 5:
                print(f"\n**After Move {move_count}:**")
                env.print_board_markdown()

            done, winner = env.is_done()
            if done:
                if winner == 1:
                    wins_x += 1
                    print("**X wins!**")
                elif winner == -1:
                    wins_o += 1
                    print("**O wins!**")
                else:
                    draws += 1
                    print("**Draw!**")

    print(f"\n### Final Results after {num_games} games vs Random O:")
    print(f"**X wins: {wins_x}**, **O wins: {wins_o}**, **Draws: {draws}**")
    win_rate = (wins_x / num_games) * 100
    print(f"**X Win Rate (Accuracy): {win_rate:.1f}%**")

# Run training and testing
if __name__ == "__main__":
    print("Training the model with epsilon decay for higher accuracy...")
    agent_x = train_model(20000)  # Train longer for better performance

    print("\nTesting the trained X agent against Random O...")
    test_model_vs_random(agent_x, 10)

Training the model with epsilon decay for higher accuracy...
Episode 0 completed. Epsilon X: 0.995, O: 0.995
Episode 5000 completed. Epsilon X: 0.010, O: 0.010
Episode 10000 completed. Epsilon X: 0.010, O: 0.010
Episode 15000 completed. Epsilon X: 0.010, O: 0.010

Testing the trained X agent against Random O...

### Game 1 vs Random O:

**After Move 1:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   |   |   |
|---|---|---|
|   |   |   |


**After Move 3:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   |   |   |
|---|---|---|
|   | O | X |


**After Move 5:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   | O | X |
|---|---|---|
|   | O | X |


**After Move 6:**
|   |   |   |
|---|---|---|
| X | O |   |
|---|---|---|
|   | O | X |
|---|---|---|
|   | O | X |

**O wins!**

### Game 2 vs Random O:

**After Move 1:**
|   |   |   |
|---|---|---|
| X |   |   |
|---|---|---|
|   |   |   |
|---|---|---|
|   |   |   |


**After Move 3:**
|   |   |   |
|-